# Livestock Analysis

Livestock analysis based on the dataframe previously created using scrape routine and OCR to extract data from auctions pdf's from local auction website.

### Import Modules

In [2]:
# Pandas
import pandas as pd
# Numpy
import numpy as np
# Import math for ceil method in calculating bin
import math
# Plotly
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True) # Offline plotting
# Plotly controls
from IPython.display import display, HTML

### Clean Up Dataframe

In [3]:
# Read dataframe
df = pd.read_csv("data.csv")
# Remove useless columns
df.drop(["Unnamed: 0", "age"], axis=1, inplace=True)
# Turn date type from objecto to datetime
df["date"] = df["date"].astype('datetime64[ns]')
# Create year, month and week columns
df["year"] = df["date"].apply(lambda x: x.year)
df["month"] = df["date"].apply(lambda x: x.month)
df["week"] = df["date"].apply(lambda x: x.week)
# Create sex column
df["sex"] = df.category.str.contains("MACHO").replace({True: 1, False: 0})
# Fill NaN values with 0 = Female. Checked manually. The ones missing are in file 17-08-31 and are all female 1st iron.
df["sex"].fillna(0, inplace=True)
# Create iron column
df["iron"] = df.category.str.contains("PRIMERA") + df.category.str.contains("1a")
# Fill NaN values with 0 = Female. Checked manually. The ones missing are in file 17-08-31 and are all female 1st iron.
df["iron"].fillna(1, inplace=True)
df["iron"] = df["iron"].replace({0: 2})
# Change types
df["sex"] = df["sex"].astype(np.int32)
df["iron"] = df["iron"].astype(np.int32)
# Drop category column
df.drop(["category"], axis=1, inplace=True)
# Check if there are no null values in sex column and in iron column
if (df["sex"].isnull().values.any() == False) and (df["iron"].isnull().values.any() == False): print("Ente gut, alles gut!")

Ente gut, alles gut!


### Sales Volumes By Units Sold and COP

Histogram showing the volumes transacted according to the weight. In this specific example we are interested in the males. Segmented by numer of units sold and amount of money transacted in the local currency: COP.

In [4]:
# Dataframe of Males: ML & MC types.
df_1 = df[(df["type"] == "ML") | (df["type"] == "MC")]

In [5]:
# Generate volume histogram by weight
def volume_hist_by_weight(dataframe):
    # Histogram dict
    weight_hist_dict = {}
    # Set bin size for weight
    bin_size = 10
    # Main routine
    for index, row in dataframe.iterrows():
        # Calculate weight bin
        weight_bin = int(bin_size*math.ceil(row["weight"]/bin_size) - (bin_size/2))
        # Update weight_hist_dict
        if weight_bin in weight_hist_dict:
            weight_hist_dict[weight_bin][0] += row["quantity"]
            weight_hist_dict[weight_bin][1] += row["quantity"]*row["price_unit"]
        else:
            weight_hist_dict[weight_bin] = [row["quantity"], row["quantity"]*row["price_unit"]]
    # Sorted keys
    sorted_keys = sorted([k for k in weight_hist_dict])
    # Data dictionary for output
    data_dict = {"unit_volume": [], "cop_volume": [], "bins": []}
    # Populate data dictionary
    for key in sorted_keys:
        data_dict["bins"].append(key)
        data_dict["unit_volume"].append(weight_hist_dict[key][0])
        data_dict["cop_volume"].append(weight_hist_dict[key][1]) 
    # Return data dict
    return data_dict

In [6]:
def plot_volumes(volume_type):
    # Define years
    years = ['2013', '2014', '2015', '2016', '2017']

    # Make figure
    figure = {
        'data': [],
        'layout': {},
        'frames': []
    }

    # Fill in most of layout
    figure['layout']['xaxis'] = {'range': [0, 700], 'title': 'Weight'}
    figure['layout']['yaxis'] = {'title': 'Volume [Units]'}
    figure['layout']['hovermode'] = 'closest'
    figure['layout']['sliders'] = {
        'args': [
            'transition', {
                'duration': 1000,
                'easing': 'cubic-in-out'
            }
        ],
        'initialValue': '2013',
        'plotlycommand': 'animate',
        'values': years,
        'visible': True
    }
    figure['layout']['updatemenus'] = [
        {
            'buttons': [
                {
                    'args': [None, {'frame': {'duration': 5000, 'redraw': True},
                             'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                    'label': 'Play',
                    'method': 'animate'
                },
                {
                    'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate',
                    'transition': {'duration': 0}}],
                    'label': 'Pause',
                    'method': 'animate'
                }
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }
    ]

    sliders_dict = {
        'active': 0,
        'yanchor': 'top',
        'xanchor': 'left',
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Year:',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {'duration': 300, 'easing': 'cubic-in-out'},
        'pad': {'b': 10, 't': 50},
        'len': 0.9,
        'x': 0.1,
        'y': 0,
        'steps': []
    }

    # Make data
    year = 2013

    # Initialize data
    df_1_year = df_1[df_1['year'] == year]
    data = volume_hist_by_weight(df_1_year)
    data_dict = go.Bar(
        x=data["bins"],
        y=data[volume_type]
    )
    figure['data'].append(data_dict)

    # Make frames
    for year in years:
        frame = {'data': [], 'name': str(year)}

        df_1_year = df_1[df_1['year'] == int(year)]
        data = volume_hist_by_weight(df_1_year)

        data_dict = go.Bar(
            x=data["bins"],
            y=data[volume_type]
        )
        frame['data'].append(data_dict)

        figure['frames'].append(frame)
        slider_step = {'args': [
            [year],
            {'frame': {'duration': 500, 'redraw': True},
             'mode': 'immediate',
           'transition': {'duration': 500}}
         ],
         'label': year,
         'method': 'animate'}
        sliders_dict['steps'].append(slider_step)

    figure['layout']['sliders'] = [sliders_dict]

    iplot(figure)

#### Sales Volume in Units

In [7]:
# Plot unit volume
plot_volumes("unit_volume")

#### Sales Volume in COP

In [8]:
# Plot COP volume
plot_volumes("cop_volume")

### What We Care About: Profit

The point is to find the parameters that give the optimal profit. There are 2 independent variables: the initial weight of the animal and the number of months is kept - this is assuming a fixed number of kilograms per day increase. 

Let:

1. $p_i$ be the purchase price.
2. $p_f$ be the selling price.
3. $\overline{kg}_i$ be the average kilograms per animal on purchase.
4. $\overline{kg}_f$ be the average kilograms per animal when selling.
5. $\Delta t$ be the time lapse in months.
6. $n$ be the number of animals bought.

Then the $ROI$ will be:

$$ROI = \frac{n\cdot (p_f\cdot \overline{kg}_f-p_i\cdot \overline{kg}_i)}{n\cdot p_i\cdot \overline{kg}_i} 
= \frac{p_f\cdot \overline{kg}_f}{p_i\cdot \overline{kg}_i}-1$$

Now, given that $\overline{kg}_f$ depends on the purchase weight and the number of months the livestock is fed, we have:

$$ROI = \frac{p_f\cdot (\overline{kg}_i+ \dot{\overline{kg}}\cdot \Delta t)}{p_i\cdot \overline{kg}_i}-1 = \frac{p_f}{p_i}\cdot \left(\frac{\dot{\overline{kg}}\cdot \Delta t}{\overline{kg}_i} +1\right)-1$$

The former equation shows the $ROI$ of the operation, where: $\dot{\overline{kg}}$ is the average weight gain per month per unit. However, this equation depends on the time lapse - number of months - of the operation. In order to have a standard measure lets look at the annualized $ROI$.

$$ROI_{ANNUALIZED} = \left[\frac{p_f}{p_i}\cdot \left(\frac{\dot{\overline{kg}}\cdot \Delta t}{\overline{kg}_i} +1\right)-1\right]^{\left(\frac{12}{\Delta t}\right)}$$

In [108]:
# Make a dataframe with index of date in the format YYYY-MM and the columns the different weight bins.
# The elements of the matrix will correspond to the price of the livestock unit for that specific time period.
def get_price_dataframe(df, livestock_types):
    # Filter condition
    filter_condition = False
    for livestock_type in livestock_types:
        filter_condition = filter_condition | (df["type"] == livestock_type)
    # Filtered dataframe
    df = df[filter_condition]
    # Select only relevant columns
    df = df[[ "year", "month", "week", "weight", "quantity", "price_avg"]]
    # Make price_avg_numerator 
    df["price_avg_numerator"] = df["quantity"]*df["price_avg"] 
    # Make bin
    bin_size = 10
    df["bin"] = df["weight"].apply(lambda weight: int(bin_size*math.ceil(weight/bin_size) - (bin_size/2)))
    # Grouped by year, month and bin
    df_grouped = df.groupby(["year", "month", "bin"])
    # Prices by year, month and weight bin
    df_prices_by_year_month_bin = pd.DataFrame([])
    df_prices_by_year_month_bin["price"] = (df_grouped["price_avg_numerator"].sum()/df_grouped["quantity"].sum()).apply(lambda x: round(x))
    # Output dataframe
    return df_prices_by_year_month_bin

In [110]:
df_roi_annualized

price  month_1  month_2  month_3  month_4  month_5  month_6  \
year month bin                                                                
2013 1     115   2012      NaN      NaN      NaN      NaN      NaN      NaN   
           125   2650      NaN      NaN      NaN      NaN      NaN      NaN   
           135   2825      NaN      NaN      NaN      NaN      NaN      NaN   
           145   2497      NaN      NaN      NaN      NaN      NaN      NaN   
           165   2546      NaN      NaN      NaN      NaN      NaN      NaN   
           175   2950      NaN      NaN      NaN      NaN      NaN      NaN   
           195   2532      NaN      NaN      NaN      NaN      NaN      NaN   
           205   2845      NaN      NaN      NaN      NaN      NaN      NaN   
           235   2642      NaN      NaN      NaN      NaN      NaN      NaN   
           245   2888      NaN      NaN      NaN      NaN      NaN      NaN   
           265   2710      NaN      NaN      NaN      NaN      NaN      NaN   
           275   2500      NaN      NaN      NaN      NaN      NaN      NaN   
           295   2690      NaN      NaN      NaN      NaN      NaN      NaN   
           305   2754      NaN      NaN      NaN      NaN      NaN      NaN   
           315   2300      NaN      NaN      NaN      NaN      NaN      NaN   
           325   2740      NaN      NaN      NaN      NaN      NaN      NaN   
           335   2533      NaN      NaN      NaN      NaN      NaN      NaN   
           345   2770      NaN      NaN      NaN      NaN      NaN      NaN   
           365   2607      NaN      NaN      NaN      NaN      NaN      NaN   
           375   2769      NaN      NaN      NaN      NaN      NaN      NaN   
           385   2711      NaN      NaN      NaN      NaN      NaN      NaN   
           395   2640      NaN      NaN      NaN      NaN      NaN      NaN   
           405   2600      NaN      NaN      NaN      NaN      NaN      NaN   
           425   2720      NaN      NaN      NaN      NaN      NaN      NaN   
           445   2668      NaN      NaN      NaN      NaN      NaN      NaN   
     2     95    2300      NaN      NaN      NaN      NaN      NaN      NaN   
           105   2433      NaN      NaN      NaN      NaN      NaN      NaN   
           115   3150      NaN      NaN      NaN      NaN      NaN      NaN   
           125   2633      NaN      NaN      NaN      NaN      NaN      NaN   
           135   2533      NaN      NaN      NaN      NaN      NaN      NaN   
...               ...      ...      ...      ...      ...      ...      ...   
2017 8     255   4596      NaN      NaN      NaN      NaN      NaN      NaN   
           265   4585      NaN      NaN      NaN      NaN      NaN      NaN   
           275   4798      NaN      NaN      NaN      NaN      NaN      NaN   
           295   4264      NaN      NaN      NaN      NaN      NaN      NaN   
           305   4353      NaN      NaN      NaN      NaN      NaN      NaN   
           325   4322      NaN      NaN      NaN      NaN      NaN      NaN   
           335   4179      NaN      NaN      NaN      NaN      NaN      NaN   
           345   4520      NaN      NaN      NaN      NaN      NaN      NaN   
           355   4500      NaN      NaN      NaN      NaN      NaN      NaN   
           365   4457      NaN      NaN      NaN      NaN      NaN      NaN   
           375   4200      NaN      NaN      NaN      NaN      NaN      NaN   
           385   4200      NaN      NaN      NaN      NaN      NaN      NaN   
           415   4080      NaN      NaN      NaN      NaN      NaN      NaN   
           425   4193      NaN      NaN      NaN      NaN      NaN      NaN   
           445   4295      NaN      NaN      NaN      NaN      NaN      NaN   
           455   3780      NaN      NaN      NaN      NaN      NaN      NaN   
     9     115   4520      NaN      NaN      NaN      NaN      NaN      NaN   
           125   5100      NaN      NaN      NaN      NaN      NaN

In [151]:
# Set variables for ROI calculations
weight_gain_per_month = 20 # Set to 20 kg per month. It will be a multiple of 10 and will jump 2 bins/month 
len_df_roi_annualized = len(df_roi_annualized)
# Get dataframe to hold ROI_annualized 
df_roi_annualized = get_price_dataframe(df, ["ML", "MC"])
# Add columns for the twelve months
for i in range(12): df_roi_annualized["month_{0}".format(i+1)] = math.nan

# Indexes of dataframe
indexes_df_roi_annualized = df_roi_annualized.index

# Loop through each row of the df_roi_annualized dataframe to populate it
for i in range(len_df_roi_annualized):
    # Get i-th row
    row_i = df_roi_annualized.iloc[i]
    # Get year, month and initial weight_bin
    year_i, month_i, weight_bin_i = row_i.name
    # Get the initial price
    price_i = row_i["price"] 
    # Calculate each ROI_annualized for the months duration for that row 
    for j in range(12):
        # Months duration
        delta_t_months = j+1
        # Final weight bin obtained in the months duration
        weight_bin_f = weight_bin_i + weight_gain_per_month * delta_t_months
        # Final date after the months duration
        year_f = year_i
        month_f = month_i + delta_t_months
        if month_f > 12: # If the month goes overboard to the next year => Recalculate year and month
            year_f = year_i + 1
            month_f = month_f - 12
        # Check that the corresponding final index exists in the dataframe, i.e. that
        # the price exists for the final values of year, month and weight
        if (year_f, month_f, weight_bin_f) in indexes_df_roi_annualized:
            # Get final price
            price_f = df_roi_annualized.loc[(year_f, month_f, weight_bin_f), "price"]
            # Calculate roi_annualized for the i-th row and column: delta_t_months
            roi_annualized_i_j = ((price_f/price_i)*((weight_gain_per_month*delta_t_months/weight_bin_i)+1)-1)**(12/delta_t_months)
            # Set roi_annualized for the i-th row and column: delta_t_months into df_roi_annualized dataframe if is more than 20%
            df_roi_annualized.loc[(year_i,month_i,weight_bin_i), "month_{0}".format(delta_t_months)] = roi_annualized_i_j
            if (year_i == 2013) & (month_i == 1) & (weight_bin_i == 245):
                print("Initial values:", (year_i, month_i, weight_bin_i))
                print("Final values:", (year_f, month_f, weight_bin_f))
                print("Months:", delta_t_months)
                print("Is in dataframe:", (year_f, month_f, weight_bin_f) in indexes_df_roi_annualized)
                print("ROI_annualized:", roi_annualized_i_j)
                print("================================================")

Initial values: (2013, 1, 245)
Final values: (2013, 2, 265)
Months: 1
Is in dataframe: True
ROI_annualized: 1.18804542657e-41
Initial values: (2013, 1, 245)
Final values: (2013, 3, 285)
Months: 2
Is in dataframe: True
ROI_annualized: 0.000184453355011
Initial values: (2013, 1, 245)
Final values: (2013, 4, 305)
Months: 3
Is in dataframe: True
ROI_annualized: 0.00210877460263
Initial values: (2013, 1, 245)
Final values: (2013, 5, 325)
Months: 4
Is in dataframe: True
ROI_annualized: 0.0501570695099
Initial values: (2013, 1, 245)
Final values: (2013, 7, 365)
Months: 6
Is in dataframe: True
ROI_annualized: 0.254778397876
Initial values: (2013, 1, 245)
Final values: (2013, 9, 405)
Months: 8
Is in dataframe: True
ROI_annualized: 0.636655230463


/Users/pedroatencio/anaconda/envs/py361/lib/python3.6/site-packages/ipykernel_launcher.py:38: RuntimeWarning:

invalid value encountered in double_scalars



In [161]:
df_roi_annualized.loc[(2016,3),:]

,price,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
bin,,,,,,,,,,,,,
105,3550,8.576390e-07,0.010414,0.668746,NaN,2.764741,0.983968,2.884715,3.039370,3.258615,3.550201,NaN,2.818833
115,3224,NaN,0.003280,0.208348,2.897701,NaN,3.365398,3.404382,3.639160,3.265703,NaN,2.804475,NaN
125,3628,1.762782e-08,0.004947,0.159290,1.116264,2.079402,1.347852,2.356688,2.198957,2.229740,2.350691,2.401508,2.219405
135,3511,1.358672e-06,0.014116,0.265808,NaN,1.588698,2.174923,1.909221,2.177489,NaN,2.359260,NaN,2.278585
155,3679,8.462969e-10,0.000102,NaN,0.567852,0.565711,NaN,1.530720,NaN,1.473686,NaN,1.329541,1.770739
165,3788,1.061700e-08,0.000515,0.097219,0.434954,0.738264,0.799528,1.087277,NaN,NaN,1.499163,NaN,1.630796
175,3550,3.295749e-08,NaN,0.038155,0.596409,0.431030,1.032681,1.443130,1.412423,1.524540,NaN,1.662434,1.729481
195,3797,1.034664e-15,0.000099,0.033846,0.278651,0.429625,NaN,0.929415,1.064428,0.822680,1.191751,1.270439,NaN
205,3459,3.077454e-09,0.000210,0.159557,0.611604,0.731648,0.729227,1.271654,1.262447,1.300662,NaN,1.479049,1.466313
